In [3]:
base_url="https://rupaulsdragrace.fandom.com"
season13_url="/wiki/RuPaul%27s_Drag_Race_(Season_13)"


In [31]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [4]:
response = requests.get(base_url+season13_url)
response

<Response [200]>

In [5]:
drag_links=[]

In [6]:
soup=bs(response.text)

In [11]:
img = soup.find_all(attrs={"data-image-key": re.compile("(\w+)QueenMug.jpg")})
hrefs = [drag.parent for drag in img]

In [14]:
drag_links= [drag["href"] for drag in hrefs]
drag_links

['/wiki/Denali',
 '/wiki/Elliott_with_2_Ts',
 '/wiki/Gottmik',
 '/wiki/Joey_Jay',
 '/wiki/Kahmora_Hall',
 '/wiki/Kandy_Muse',
 '/wiki/LaLa_Ri',
 '/wiki/Olivia_Lux',
 '/wiki/Ros%C3%A9',
 '/wiki/Symone',
 '/wiki/Tamisha_Iman',
 '/wiki/Tina_Burner',
 '/wiki/Utica_Queen']

In [47]:
linkdenali = ""
drags=[]
for drag_link in drag_links:
    temp_soup = bs(requests.get(base_url + drag_link).text)
    
    temp_name=(temp_soup.find(attrs={"data-source": "Drag Name"}).find("div").text)
    temp_age=(temp_soup.find_all(attrs={"data-source": "birth year"})[1].find("div").text)
    try:
        temp_city=(temp_soup.find(attrs={"data-source": "Current City"}).find("div").text)
    except AttributeError:
        temp_city="Unknown"
    temp_place=((temp_soup.find(attrs={"data-source": "Place"}).find("div").text)[0])
    temp_eliminated=temp_soup.find(attrs={"data-source": "Eliminated"}).find("div").text
    temp_wins=((temp_soup.find(attrs={"data-source": "Challenge Wins"}).find("div").text)[0])
    temp_bottoms=((temp_soup.find_all("tr")[3].find("td").text)[0])
    temp_tw_handler=(re.match("https://twitter.com/(\w+)" ,\
        temp_soup.find(attrs={"href": re.compile("https://twitter.com/(\w+)")})["href"]).group(1))
    print(temp_tw_handler)
    temp_id=input()
    drags.append((temp_id, temp_name, temp_tw_handler, temp_age, temp_city, temp_place, temp_eliminated, temp_wins, temp_bottoms))

denalifox


 1151340112955019265


theelliottqueen


 2274214254


gottmik


...

 951228252747689991


thetinaburner


 874256563


QueenUtica


 1105838381622329344


In [1]:
df = pd.DataFrame(drags, columns = ["ID", "Name", "Twitter_handler", "Age", "City", "Place", "Episode_eliminated", "Wins", "Bottoms"])

NameError: name 'pd' is not defined

In [2]:
df["Place"]=[ fixPlace(base_url + drag_link) for drag_link in drag_links]
df

NameError: name 'drag_links' is not defined

In [56]:
def fixPlace(link):
    soup = bs(requests.get(link).text)
    place = soup.find(attrs={"data-source": "Place"}).find("div").text
    if place[0] == "R":
        place="2"
    elif place[0] == "W":
        place="1"
    
    return re.match("([0-9]+)", place).group(1)

In [58]:
df.to_csv('Data/drags.csv',index=False)

In [8]:
db_score_url='/wiki/"Dusted_or_Busted"_Scores/RuPaul%27s_Drag_Race#Season_13'

In [9]:
response = requests.get(base_url+db_score_url)
response

<Response [200]>

In [10]:
soup = bs(response.text)

In [11]:
table = soup.find_all("table", class_= "wikitable sortable")[12]
rows = table.find("tbody").find_all("tr")
rows.pop(0)
rows[2].find_all("td")[16].text.strip("\n")
#scores = [ row.find_all("td")[15].text.strip("\n") for row in rows]
#scores

'6.364'

In [12]:
df = pd.read_csv("Data/drags.csv")
df

,ID,Name,Twitter_handler,Age,City,Place,Episode_eliminated,Wins,Bottoms
0,1151340112955019265,Denali Foxx,denalifox,29,"Chicago, IL, USA",8,Episode 10,1,2
1,2274214254,Elliott with 2 Ts,theelliottqueen,27,"Las Vegas, NV, USA",9,Episode 9,0,0
2,1331018240470917120,Gottmik,gottmik,24,"Los Angeles, CA, USA",3,Episode 16,2,1
3,26068447,Joey Jay,joeyjayisgay,30,"Phoenix, AZ, USA",12,Episode 5,0,0
4,635906376,Kahmora Hall-Iman,KahmoraHall,29,Unknown,13,Episode 4,0,0
5,89282580,Kandy Muse,TheKandyMuse,26,"Los Angeles, CA, USA",2,Episode 16,1,2
6,1198827021104750592,LaLa Ri,misslalari,30,Unknown,10,Episode 7,0,1
7,1304051396942139397,Olivia Lux,TheOliviaLux,27,"New York City, NY, USA",5,Episode 13,2,1
8,967299284235440128,Rosé,omgheyrose,31,"New York City, NY, USA",3,Episode 16,3,1
9,1218767468954411011,Symone,the_symonee,26,"Los Angeles, CA, USA",1,NaN,4,2


In [27]:
drag_names = list(df["Name"])
#print(drag_names[0])
#denali = list(filter(lambda v: (v.text in drag_name) and v.text!="", table.find_all("a")))[0]
#score = list(filter(lambda v: re.search("\d+\.\d+",v.text),denali.parent.parent.parent.find_all("td")))[0].text.strip("\n")
scores = []
for drag_name in drag_names:
    print(drag_name)
    drag = list(filter(lambda v: (v.text.replace(" ", "") in drag_name.replace(" ", "")) and v.text!="", table.find_all("a")))[0]
    scores.append(list(filter(lambda v: re.search("\d+\.\d{3}",v.text),drag.parent.parent.parent.find_all("td")))[0].text.strip("\n"))
len(scores)

Denali Foxx
Elliott with 2 Ts
Gottmik
Joey Jay
Kahmora Hall-Iman
Kandy Muse
LaLa Ri
Olivia Lux
Rosé
Symone
Tamisha Iman
Tina Burner
Utica Queen


13

In [29]:
df["DB_scores"]=scores
df

,ID,Name,Twitter_handler,Age,City,Place,Episode_eliminated,Wins,Bottoms,DB_scores
0,1151340112955019265,Denali Foxx,denalifox,29,"Chicago, IL, USA",8,Episode 10,1,2,5.000
1,2274214254,Elliott with 2 Ts,theelliottqueen,27,"Las Vegas, NV, USA",9,Episode 9,0,0,4.285
2,1331018240470917120,Gottmik,gottmik,24,"Los Angeles, CA, USA",3,Episode 16,2,1,6.364
3,26068447,Joey Jay,joeyjayisgay,30,"Phoenix, AZ, USA",12,Episode 5,0,0,2.500
4,635906376,Kahmora Hall-Iman,KahmoraHall,29,Unknown,13,Episode 4,0,0,1.250
5,89282580,Kandy Muse,TheKandyMuse,26,"Los Angeles, CA, USA",2,Episode 16,1,2,4.773
6,1198827021104750592,LaLa Ri,misslalari,30,Unknown,10,Episode 7,0,1,3.000
7,1304051396942139397,Olivia Lux,TheOliviaLux,27,"New York City, NY, USA",5,Episode 13,2,1,4.659
8,967299284235440128,Rosé,omgheyrose,31,"New York City, NY, USA",3,Episode 16,3,1,7.386
9,1218767468954411011,Symone,the_symonee,26,"Los Angeles, CA, USA",1,NaN,4,2,6.364


In [36]:
df = pd.read_csv("Data/drags.csv")
df["ID"]=[str(idi) +  "" for idi in list(df["ID"])]

In [37]:
df.to_csv('Data/drags.csv',index=False)